In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import yaml
import pandas as pd

In [ ]:
import os
os.getcwd() 

# Pre-defined funciton

In [ ]:
def extract_energy_breakdown(acclg_E):
    energy_components = acclg_E['energy_estimation']['components']

    energy_breakdown= {}
    glb_count = 0
    spad_count = 0
    mac_count = 0
    dram_count = 0

    for ec in energy_components:
        skip_rest = False

        # check glb (pJ)
        for glb in ['ifmap_glb', 'weights_glb', 'psum_glb']:
            if glb in ec['name']:
                assert (glb not in energy_breakdown)
                glb_count += 1
                energy_breakdown[glb] = ec['energy']
                skip_rest = True
                break
        if skip_rest: continue

        # check spad (pJ)
        for spad in ['ifmap_spad', 'weights_spad', 'psum_spad']:
            if spad in ec['name']:
                spad_count += 1
                if spad not in energy_breakdown:
                    energy_breakdown[spad] = ec['energy']
                else:
                    energy_breakdown[spad] = energy_breakdown[spad] + ec['energy']
                skip_rest = True
   
                break
        if skip_rest: continue

        # check mac (pJ)
        if 'mac' in ec['name']:
            mac_count += 1
            if 'mac' not in energy_breakdown:
                energy_breakdown['mac'] = ec['energy']
            else:
                energy_breakdown['mac'] = energy_breakdown['mac'] + ec['energy']
        if skip_rest: continue

        # check dram (nJ)
        for dram in ['ifmap_dram', 'weights_dram', 'psum_dram']:
            if dram in ec['name']:
                assert (dram not in energy_breakdown)
                dram_count += 1
                energy_breakdown[dram] = ec['energy'] * 1000
                skip_rest = True
                break

    return energy_breakdown

# Single Run 

In [ ]:
### CHANGE THE RUN NAME (from scale.cfg) ###
run_name = "scale_resnet_conv_run_32x32_os"
acclg_outdir = "./output/accelergy_output_"+run_name+"/energy_estimation.yaml"
scsim_outdir = "./output/scale_sim_output_"+run_name+"/COMPUTE_REPORT.csv"
############################################


# Load accelergy_energy results
with open(acclg_outdir, 'r') as fr:
    acclg_E = yaml.safe_load(fr)

# Load scale-sim results
csv_filename = scsim_outdir
scsim_P = pd.read_csv(csv_filename, sep=r'\s*,\s*', engine='python')


energy_breakdown_32x32_os = extract_energy_breakdown(acclg_E)
layer_utilization = (scsim_P['Compute Util %']).values.tolist()
layer_runtime = scsim_P['Total Cycles'].values.tolist()



In [ ]:
labels_32x32_os = []
values_32x32_os = []
for k,v in energy_breakdown_32x32_os.items():
    labels_32x32_os.append(k)
    values_32x32_os.append(v/1000000)
    print(k,v/1000000)

values = [list(x) for x in zip(values_32x32_os)]
    
ax = plt.figure(figsize=(12,5))
bar_plot = plt.barh(['32x32_os'], values[0])
left_coor = values[0]

for n in range(1,len(values)):
    plt.barh(['32x32_os'], values[n], left=left_coor)
    left_coor = [x1+x2 for x1,x2 in zip(left_coor, values[n])]
    
plt.title(run_name)    
plt.legend(labels_32x32_os, ncol=4, bbox_to_anchor=(0.98, -0.15), fontsize=13)
plt.xlabel('Energy (uJ)', fontsize=15)

plt.yticks(fontsize=15)
plt.xticks(fontsize=12)

plt.show()

print("Runtime for each layer (cycles)", layer_runtime)
print("Utilizations for each layer (%)", layer_utilization)

# For Report (Outdated, TO BE FIXED)

## Different Sizes


In [ ]:
# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_32x32_os/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_32x32_os = extract_energy_breakdown(acclg_E)

# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_64x64_os/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_64x64_os = extract_energy_breakdown(acclg_E)

# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_16x16_os/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_16x16_os = extract_energy_breakdown(acclg_E)



In [ ]:
labels_16x16_os = []
values_16x16_os = []
for k,v in energy_breakdown_16x16_os.items():
    labels_16x16_os.append(k)
    values_16x16_os.append(v/1000000)

labels_32x32_os = []
values_32x32_os = []
for k,v in energy_breakdown_32x32_os.items():
    labels_32x32_os.append(k)
    values_32x32_os.append(v/1000000)

labels_64x64_os = []
values_64x64_os = []
for k,v in energy_breakdown_64x64_os.items():
    labels_64x64_os.append(k)
    values_64x64_os.append(v/1000000)   

    
values = [list(x) for x in zip(values_16x16_os, values_32x32_os, values_64x64_os)]


ax = plt.figure(figsize=(10,5))
bar_plot = plt.barh(['16x16_os', '32x32_os', '64x64_os'], values[0])
left_coor = values[0]

for n in range(1,7):
    plt.barh(['16x16_os', '32x32_os', '64x64_os'], values[n], left=left_coor)
    left_coor = [x1+x2 for x1,x2 in zip(left_coor, values[n])]
    
    
plt.legend(labels_32x32_os, ncol=4, bbox_to_anchor=(0.98, -0.15), fontsize=13)
plt.xlabel('Energy (uJ)', fontsize=15)

plt.yticks(fontsize=15)
plt.xticks(fontsize=12)

plt.show()

## Different Dataflows

In [ ]:
# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_32x32_is/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_32x32_is = extract_energy_breakdown(acclg_E)

# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_32x32_ws/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_32x32_ws = extract_energy_breakdown(acclg_E)

# Load accelergy_energy estimator
with open("./output/accelergy_output/scale_resnet_conv_run_32x32_os/energy_estimation.yaml", 'r') as fr:
    acclg_E = yaml.safe_load(fr)

energy_breakdown_32x32_os = extract_energy_breakdown(acclg_E)

In [ ]:
labels_32x32_is = []
values_32x32_is = []
for k,v in energy_breakdown_32x32_is.items():
    labels_32x32_is.append(k)
    values_32x32_is.append(v/1000000)

labels_32x32_ws = []
values_32x32_ws = []
for k,v in energy_breakdown_32x32_ws.items():
    labels_32x32_ws.append(k)
    values_32x32_ws.append(v/1000000)

labels_32x32_os = []
values_32x32_os = []
for k,v in energy_breakdown_32x32_os.items():
    labels_32x32_os.append(k)
    values_32x32_os.append(v/1000000)
    
values = [list(x) for x in zip(values_32x32_is, values_32x32_ws, values_32x32_os)]


ax = plt.figure(figsize=(10,5))
bar_plot = plt.barh(['32x32_is', '32x32_ws', '32x32_os'], values[0])
left_coor = values[0]


for n in range(1,7):
    plt.barh(['32x32_is', '32x32_ws', '32x32_os'], values[n], left=left_coor)
    left_coor = [x1+x2 for x1,x2 in zip(left_coor, values[n])]

    
plt.yticks(fontsize=15)
plt.xticks(fontsize=12)

plt.legend(labels_32x32_ws, ncol=4, bbox_to_anchor=(0.98, -0.15),fontsize=13)
plt.xlabel('Energy (uJ)', fontsize=15)
plt.show()